# Setup

In [ ]:
ALTER ACCOUNT SET TIMEZONE = 'Asia/Ho_Chi_Minh';
SHOW PARAMETERS LIKE 'TIMEZONE';

# Roles-Based Access Control (RBAC)

In [ ]:
USE ROLE USERADMIN;

/*
TPCH_ADMIN;
TPCH_DEVELOPER; -- Developer: load data, transform
TPCH_ANALYST; -- Analyst: query, report
TPCH_VIEWER; -- Viewer: only view reports
*/

CREATE ROLE TPCH_ADMIN;
CREATE ROLE TPCH_DEVELOPER;
CREATE ROLE TPCH_ANALYST;
CREATE ROLE TPCH_VIEWER;

-- setup role hỉerarchy
/*
Naive Rule:
TPCH_ADMIN inherits all perms of TPCH_DEVELOPER, TPCH_DEVELOPER inherits all perms of TPCH_ANALYST, etc
*/

GRANT ROLE TPCH_VIEWER TO ROLE TPCH_ANALYST;
GRANT ROLE TPCH_ANALYST TO ROLE TPCH_DEVELOPER;
GRANT ROLE TPCH_DEVELOPER TO ROLE TPCH_ADMIN;
GRANT ROLE TPCH_ADMIN TO ROLE SYSADMIN;

# Create DB and schema

In [ ]:
USE ROLE SYSADMIN;
CREATE DATABASE TPCH_ANALYTICS_DB;

USE DATABASE TPCH_ANALYTICS_DB;
CREATE SCHEMA STAGING;
CREATE SCHEMA ANALYTICS;
CREATE SCHEMA REPORTS;
CREATE SCHEMA UDFS;
CREATE SCHEMA CONTROL;


In [ ]:
-- USE ROLE SYSADMIN;

-- REVOKE USAGE ON DATABASE TPCH_ANALYTICS_DB FROM ROLE TPCH_VIEWER;
-- REVOKE USAGE ON DATABASE TPCH_ANALYTICS_DB FROM ROLE TPCH_ANALYST;
-- REVOKE USAGE ON DATABASE TPCH_ANALYTICS_DB FROM ROLE TPCH_DEVELOPER;
-- REVOKE USAGE ON DATABASE TPCH_ANALYTICS_DB FROM ROLE TPCH_ADMIN;

-- REVOKE ALL PRIVILEGES ON SCHEMA TPCH_ANALYTICS_DB.STAGING FROM ROLE TPCH_DEVELOPER;

-- REVOKE ALL PRIVILEGES ON SCHEMA TPCH_ANALYTICS_DB.ANALYTICS FROM ROLE TPCH_ANALYST;
-- REVOKE ALL PRIVILEGES ON SCHEMA TPCH_ANALYTICS_DB.ANALYTICS FROM ROLE TPCH_DEVELOPER;

-- REVOKE ALL PRIVILEGES ON SCHEMA TPCH_ANALYTICS_DB.REPORTS FROM ROLE TPCH_VIEWER;
-- REVOKE ALL PRIVILEGES ON SCHEMA TPCH_ANALYTICS_DB.REPORTS FROM ROLE TPCH_ANALYST;
-- REVOKE ALL PRIVILEGES ON SCHEMA TPCH_ANALYTICS_DB.REPORTS FROM ROLE TPCH_DEVELOPER;

-- REVOKE ALL PRIVILEGES ON SCHEMA TPCH_ANALYTICS_DB.UDFS FROM ROLE TPCH_VIEWER;
-- REVOKE ALL PRIVILEGES ON SCHEMA TPCH_ANALYTICS_DB.UDFS FROM ROLE TPCH_ANALYST;
-- REVOKE ALL PRIVILEGES ON SCHEMA TPCH_ANALYTICS_DB.UDFS FROM ROLE TPCH_DEVELOPER;

-- REVOKE SELECT, INSERT, UPDATE, DELETE ON FUTURE TABLES IN SCHEMA TPCH_ANALYTICS_DB.STAGING FROM ROLE TPCH_DEVELOPER;

-- -- ANALYTICS
-- REVOKE SELECT, INSERT, UPDATE, DELETE ON FUTURE TABLES IN SCHEMA TPCH_ANALYTICS_DB.ANALYTICS FROM ROLE TPCH_DEVELOPER;
-- REVOKE SELECT ON FUTURE TABLES IN SCHEMA TPCH_ANALYTICS_DB.ANALYTICS FROM ROLE TPCH_ANALYST;

-- -- REPORTS
-- REVOKE SELECT ON FUTURE TABLES IN SCHEMA TPCH_ANALYTICS_DB.REPORTS FROM ROLE TPCH_VIEWER;
-- REVOKE SELECT ON FUTURE TABLES IN SCHEMA TPCH_ANALYTICS_DB.REPORTS FROM ROLE TPCH_ANALYST;
-- REVOKE SELECT, INSERT, UPDATE, DELETE ON FUTURE TABLES IN SCHEMA TPCH_ANALYTICS_DB.REPORTS FROM ROLE TPCH_DEVELOPER;

-- REVOKE SELECT ON FUTURE VIEWS IN SCHEMA TPCH_ANALYTICS_DB.REPORTS FROM ROLE TPCH_VIEWER;
-- REVOKE SELECT ON FUTURE VIEWS IN SCHEMA TPCH_ANALYTICS_DB.REPORTS FROM ROLE TPCH_ANALYST;

In [ ]:
GRANT USAGE ON DATABASE TPCH_ANALYTICS_DB TO ROLE TPCH_VIEWER;

GRANT USAGE ON SCHEMA TPCH_ANALYTICS_DB.UDFS TO ROLE TPCH_VIEWER;
GRANT USAGE ON SCHEMA TPCH_ANALYTICS_DB.STAGING TO ROLE TPCH_VIEWER;
GRANT USAGE ON SCHEMA TPCH_ANALYTICS_DB.ANALYTICS TO ROLE TPCH_VIEWER;
GRANT USAGE ON SCHEMA TPCH_ANALYTICS_DB.REPORTS TO ROLE TPCH_VIEWER;
GRANT USAGE ON SCHEMA TPCH_ANALYTICS_DB.CONTROL TO ROLE TPCH_ANALYST;


In [ ]:
-- A. TPCH_VIEWER: View Reports
GRANT SELECT ON ALL TABLES IN SCHEMA TPCH_ANALYTICS_DB.REPORTS TO ROLE TPCH_VIEWER;
GRANT SELECT ON FUTURE TABLES IN SCHEMA TPCH_ANALYTICS_DB.REPORTS TO ROLE TPCH_VIEWER;
GRANT SELECT ON ALL VIEWS IN SCHEMA TPCH_ANALYTICS_DB.REPORTS TO ROLE TPCH_VIEWER;
GRANT SELECT ON FUTURE VIEWS IN SCHEMA TPCH_ANALYTICS_DB.REPORTS TO ROLE TPCH_VIEWER;


-- B. TPCH_ANALYST: inherit from TPCH_VIEWER + new perms (create view)
GRANT SELECT ON ALL TABLES IN SCHEMA TPCH_ANALYTICS_DB.ANALYTICS TO ROLE TPCH_ANALYST;
GRANT SELECT ON FUTURE TABLES IN SCHEMA TPCH_ANALYTICS_DB.ANALYTICS TO ROLE TPCH_ANALYST;
GRANT SELECT ON ALL VIEWS IN SCHEMA TPCH_ANALYTICS_DB.ANALYTICS TO ROLE TPCH_ANALYST;
GRANT SELECT ON FUTURE VIEWS IN SCHEMA TPCH_ANALYTICS_DB.ANALYTICS TO ROLE TPCH_ANALYST;

GRANT CREATE VIEW ON SCHEMA TPCH_ANALYTICS_DB.ANALYTICS TO ROLE TPCH_ANALYST;
GRANT CREATE VIEW ON SCHEMA TPCH_ANALYTICS_DB.REPORTS TO ROLE TPCH_ANALYST;

-- C. TPCH_DEVELOPER: inherit from TPCH_ANALYST + new perms (DDL/DML + control task/function)
GRANT CREATE STAGE, CREATE VIEW ON SCHEMA TPCH_ANALYTICS_DB.STAGING TO ROLE TPCH_DEVELOPER;
GRANT SELECT, INSERT, UPDATE, DELETE ON ALL TABLES IN SCHEMA TPCH_ANALYTICS_DB.STAGING TO ROLE TPCH_DEVELOPER;
GRANT SELECT, INSERT, UPDATE, DELETE ON FUTURE TABLES IN SCHEMA TPCH_ANALYTICS_DB.STAGING TO ROLE TPCH_DEVELOPER;

GRANT INSERT, UPDATE, DELETE ON ALL TABLES IN SCHEMA TPCH_ANALYTICS_DB.ANALYTICS TO ROLE TPCH_DEVELOPER;
GRANT INSERT, UPDATE, DELETE ON ALL TABLES IN SCHEMA TPCH_ANALYTICS_DB.REPORTS TO ROLE TPCH_DEVELOPER;
GRANT INSERT, UPDATE, DELETE ON FUTURE TABLES IN SCHEMA TPCH_ANALYTICS_DB.ANALYTICS TO ROLE TPCH_DEVELOPER;
GRANT INSERT, UPDATE, DELETE ON FUTURE TABLES IN SCHEMA TPCH_ANALYTICS_DB.REPORTS TO ROLE TPCH_DEVELOPER;


GRANT CREATE TABLE, CREATE PROCEDURE, CREATE TASK ON SCHEMA TPCH_ANALYTICS_DB.ANALYTICS TO ROLE TPCH_DEVELOPER;
GRANT CREATE TABLE, CREATE PROCEDURE, CREATE TASK ON SCHEMA TPCH_ANALYTICS_DB.REPORTS TO ROLE TPCH_DEVELOPER;
GRANT CREATE TABLE, CREATE PROCEDURE, CREATE TASK ON SCHEMA TPCH_ANALYTICS_DB.STAGING TO ROLE TPCH_DEVELOPER;

GRANT CREATE TABLE, CREATE PROCEDURE, CREATE TASK ON SCHEMA TPCH_ANALYTICS_DB.CONTROL TO ROLE TPCH_DEVELOPER;
GRANT CREATE FUNCTION, CREATE PIPE, CREATE TABLE, CREATE STREAM ON SCHEMA TPCH_ANALYTICS_DB.CONTROL TO ROLE TPCH_DEVELOPER;
GRANT OPERATE ON ALL TASKS IN SCHEMA TPCH_ANALYTICS_DB.CONTROL TO ROLE TPCH_DEVELOPER;
GRANT USAGE ON ALL FUNCTIONS IN SCHEMA TPCH_ANALYTICS_DB.CONTROL TO ROLE TPCH_DEVELOPER;
GRANT SELECT ON ALL STREAMS IN SCHEMA TPCH_ANALYTICS_DB.CONTROL TO ROLE TPCH_DEVELOPER;

GRANT OPERATE ON FUTURE TASKS IN SCHEMA TPCH_ANALYTICS_DB.CONTROL TO ROLE TPCH_DEVELOPER;
GRANT USAGE ON FUTURE FUNCTIONS IN SCHEMA TPCH_ANALYTICS_DB.CONTROL TO ROLE TPCH_DEVELOPER;
GRANT SELECT ON FUTURE STREAMS IN SCHEMA TPCH_ANALYTICS_DB.CONTROL TO ROLE TPCH_DEVELOPER;


GRANT CREATE FUNCTION ON SCHEMA TPCH_ANALYTICS_DB.UDFS TO ROLE TPCH_DEVELOPER;
GRANT USAGE ON ALL FUNCTIONS IN SCHEMA TPCH_ANALYTICS_DB.UDFS TO ROLE TPCH_DEVELOPER;
GRANT USAGE ON FUTURE FUNCTIONS IN SCHEMA TPCH_ANALYTICS_DB.UDFS TO ROLE TPCH_DEVELOPER;

In [ ]:
GRANT SELECT, INSERT, UPDATE, DELETE ON ALL TABLES IN SCHEMA TPCH_ANALYTICS_DB.CONTROL TO ROLE TPCH_DEVELOPER;
GRANT SELECT, INSERT, UPDATE, DELETE ON FUTURE TABLES IN SCHEMA TPCH_ANALYTICS_DB.CONTROL TO ROLE TPCH_DEVELOPER;

In [ ]:
SHOW GRANTS TO ROLE TPCH_DEVELOPER;

# Create Stage

In [ ]:
USE DATABASE TPCH_ANALYTICS_DB;
USE SCHEMA STAGING;


CREATE OR REPLACE STAGE TPCH_DATA_STAGE
    COMMENT = 'Stage contains raw TPC-H data.';

-- grant perms
GRANT READ ON STAGE TPCH_ANALYTICS_DB.STAGING.TPCH_DATA_STAGE TO ROLE TPCH_DEVELOPER;
GRANT WRITE ON STAGE TPCH_ANALYTICS_DB.STAGING.TPCH_DATA_STAGE TO ROLE TPCH_DEVELOPER;


# Create Raw table

In [ ]:
USE ROLE TPCH_ADMIN;
USE SCHEMA STAGING;

-- Bảng 1: REGION
CREATE OR REPLACE TABLE REGION (
    R_REGIONKEY NUMBER(38,0),
    R_NAME VARCHAR(25),
    R_COMMENT VARCHAR(152)
);

-- Bảng 2: NATION
CREATE OR REPLACE TABLE NATION (
    N_NATIONKEY NUMBER(38,0),
    N_NAME VARCHAR(25),
    N_REGIONKEY NUMBER(38,0),
    N_COMMENT VARCHAR(152)
);

-- Bảng 3: CUSTOMER
CREATE OR REPLACE TABLE CUSTOMER (
    C_CUSTKEY NUMBER(38,0),
    C_NAME VARCHAR(25),
    C_ADDRESS VARCHAR(40),
    C_NATIONKEY NUMBER(38,0),
    C_PHONE VARCHAR(15),
    C_ACCTBAL NUMBER(12,2),
    C_MKTSEGMENT VARCHAR(10),
    C_COMMENT VARCHAR(117)
);

-- Bảng 4: SUPPLIER
CREATE OR REPLACE TABLE SUPPLIER (
    S_SUPPKEY NUMBER(38,0),
    S_NAME VARCHAR(25),
    S_ADDRESS VARCHAR(40),
    S_NATIONKEY NUMBER(38,0),
    S_PHONE VARCHAR(15),
    S_ACCTBAL NUMBER(12,2),
    S_COMMENT VARCHAR(101)
);

-- Bảng 5: PART
CREATE OR REPLACE TABLE PART (
    P_PARTKEY NUMBER(38,0),
    P_NAME VARCHAR(55),
    P_MFGR VARCHAR(25),
    P_BRAND VARCHAR(10),
    P_TYPE VARCHAR(25),
    P_SIZE NUMBER(38,0),
    P_CONTAINER VARCHAR(10),
    P_RETAILPRICE NUMBER(12,2),
    P_COMMENT VARCHAR(23)
);

-- Bảng 6: PARTSUPP
CREATE OR REPLACE TABLE PARTSUPP (
    PS_PARTKEY NUMBER(38,0),
    PS_SUPPKEY NUMBER(38,0),
    PS_AVAILQTY NUMBER(38,0),
    PS_SUPPLYCOST NUMBER(12,2),
    PS_COMMENT VARCHAR(199)
);

-- Bảng 7: ORDERS
CREATE OR REPLACE TABLE ORDERS (
    O_ORDERKEY NUMBER(38,0),
    O_CUSTKEY NUMBER(38,0),
    O_ORDERSTATUS VARCHAR(1),
    O_TOTALPRICE NUMBER(12,2),
    O_ORDERDATE DATE,
    O_ORDERPRIORITY VARCHAR(15),
    O_CLERK VARCHAR(15),
    O_SHIPPRIORITY NUMBER(38,0),
    O_COMMENT VARCHAR(79)
);

-- Bảng 8: LINEITEM
CREATE OR REPLACE TABLE LINEITEM (
    L_ORDERKEY NUMBER(38,0),
    L_PARTKEY NUMBER(38,0),
    L_SUPPKEY NUMBER(38,0),
    L_LINENUMBER NUMBER(38,0),
    L_QUANTITY NUMBER(12,2),
    L_EXTENDEDPRICE NUMBER(12,2),
    L_DISCOUNT NUMBER(12,2),
    L_TAX NUMBER(12,2),
    L_RETURNFLAG VARCHAR(1),
    L_LINESTATUS VARCHAR(1),
    L_SHIPDATE DATE,
    L_COMMITDATE DATE,
    L_RECEIPTDATE DATE,
    L_SHIPINSTRUCT VARCHAR(25),
    L_SHIPMODE VARCHAR(10),
    L_COMMENT VARCHAR(44)
);

# Load file into table

In [ ]:
USE ROLE TPCH_DEVELOPER;

USE DATABASE TPCH_ANALYTICS_DB;
USE SCHEMA STAGING;

COPY INTO LINEITEM
FROM @TPCH_DATA_STAGE
PATTERN='.*tpch_sf1_line_item_.*\.csv'
FILE_FORMAT = (
    TYPE = CSV,             
    FIELD_DELIMITER = ',',  
    SKIP_HEADER = 1,        
    TRIM_SPACE = TRUE,  
    FIELD_OPTIONALLY_ENCLOSED_BY = '"'
    EMPTY_FIELD_AS_NULL = TRUE
) ON_ERROR = 'ABORT_STATEMENT';


COPY INTO CUSTOMER
FROM @TPCH_DATA_STAGE
PATTERN='.*tpch_sf1_customer.*\.csv'
FILE_FORMAT = (
    TYPE = CSV,             
    FIELD_DELIMITER = ',',  
    SKIP_HEADER = 1,        
    TRIM_SPACE = TRUE,  
    FIELD_OPTIONALLY_ENCLOSED_BY = '"'
    EMPTY_FIELD_AS_NULL = TRUE
) ON_ERROR = 'ABORT_STATEMENT';

COPY INTO NATION
FROM @TPCH_DATA_STAGE
PATTERN='.*tpch_sf1_nation.*\.csv'
FILE_FORMAT = (
    TYPE = CSV,             
    FIELD_DELIMITER = ',',  
    SKIP_HEADER = 1,        
    TRIM_SPACE = TRUE,  
    FIELD_OPTIONALLY_ENCLOSED_BY = '"'
    EMPTY_FIELD_AS_NULL = TRUE
) ON_ERROR = 'ABORT_STATEMENT';

COPY INTO ORDERS
FROM @TPCH_DATA_STAGE
PATTERN='.*tpch_sf1_orders.*\.csv'
FILE_FORMAT = (
    TYPE = CSV,             
    FIELD_DELIMITER = ',',  
    SKIP_HEADER = 1,        
    TRIM_SPACE = TRUE,  
    FIELD_OPTIONALLY_ENCLOSED_BY = '"'
    EMPTY_FIELD_AS_NULL = TRUE
) ON_ERROR = 'ABORT_STATEMENT';


COPY INTO PART
FROM @TPCH_DATA_STAGE
PATTERN='.*tpch_sf1_part.csv'
FILE_FORMAT = (
    TYPE = CSV,             
    FIELD_DELIMITER = ',',  
    SKIP_HEADER = 1,        
    TRIM_SPACE = TRUE,  
    FIELD_OPTIONALLY_ENCLOSED_BY = '"'
    EMPTY_FIELD_AS_NULL = TRUE
) ON_ERROR = 'ABORT_STATEMENT';

COPY INTO PARTSUPP
FROM @TPCH_DATA_STAGE
PATTERN='.*tpch_sf1_partsupp.*\.csv'
FILE_FORMAT = (
    TYPE = CSV,             
    FIELD_DELIMITER = ',',  
    SKIP_HEADER = 1,        
    TRIM_SPACE = TRUE,  
    FIELD_OPTIONALLY_ENCLOSED_BY = '"'
    EMPTY_FIELD_AS_NULL = TRUE
) ON_ERROR = 'ABORT_STATEMENT';

COPY INTO REGION
FROM @TPCH_DATA_STAGE
PATTERN='.*tpch_sf1_region.*\.csv'
FILE_FORMAT = (
    TYPE = CSV,             
    FIELD_DELIMITER = ',',  
    SKIP_HEADER = 1,        
    TRIM_SPACE = TRUE,  
    FIELD_OPTIONALLY_ENCLOSED_BY = '"'
    EMPTY_FIELD_AS_NULL = TRUE
) ON_ERROR = 'ABORT_STATEMENT';

COPY INTO SUPPLIER
FROM @TPCH_DATA_STAGE
PATTERN='.*tpch_sf1_supplier.*\.csv'
FILE_FORMAT = (
    TYPE = CSV,             
    FIELD_DELIMITER = ',',  
    SKIP_HEADER = 1,        
    TRIM_SPACE = TRUE,  
    FIELD_OPTIONALLY_ENCLOSED_BY = '"'
    EMPTY_FIELD_AS_NULL = TRUE
) ON_ERROR = 'ABORT_STATEMENT';


# End